# Import & summarize data

In [1]:
import pandas as pd
import numpy as np
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [2]:
train = pd.read_csv('../geotab-data/train.csv')
test = pd.read_csv('../geotab-data/test.csv')

In [53]:
print("Train dataset shape: "+ str(train.shape))
print("Test dataset shape:  "+ str(test.shape))

Train dataset shape: (857409, 28)
Test dataset shape:  (1920335, 13)


In [13]:
#below I loop through columsn and get distinct values within each column for both the train and test sets
cols = list(train.columns)
info = []
for x in cols:
    distinct = len(train[x].drop_duplicates())
    d_type = type(train[x][0])
    info.append([distinct, d_type])
train_info = pd.DataFrame(dict(zip(cols,info)), index=['distinct values', 'type']).T.reset_index() 
cols = list(test.columns)
info = []
for x in cols:
    distinct = len(test[x].drop_duplicates())
    d_type = type(test[x][0])
    info.append([distinct, d_type])
test_info = pd.DataFrame(dict(zip(cols,info)), index=['distinct values', 'type']).T.reset_index() 

In [51]:
#here I merge the dataframe and do an inner join to drop the columns not in test (will analyze seperately)
trainvtest = pd.merge(train_info, test_info, on='index', how='inner', suffixes=(' train', ' test'))
trainvtest = trainvtest.drop('type train', axis=1) #drop because types are the same

#get the difference between distinct values, a lot more in the test set
trainvtest['diff'] = trainvtest['distinct values train'] - trainvtest['distinct values test']
trainvtest.columns = ['column', 'distinct values train', 'distinct values test', 'type', 'diff']
trainvtest[['column', 'distinct values train', 'distinct values test','diff', 'type']]

,column,distinct values train,distinct values test,diff,type
0,RowId,857409,1920335,-1062926,<class 'numpy.int64'>
1,IntersectionId,2539,2768,-229,<class 'numpy.int64'>
2,Latitude,4505,5613,-1108,<class 'numpy.float64'>
3,Longitude,4541,5614,-1073,<class 'numpy.float64'>
4,EntryStreetName,1708,2137,-429,<class 'str'>
5,ExitStreetName,1694,2118,-424,<class 'str'>
6,EntryHeading,8,8,0,<class 'str'>
7,ExitHeading,8,8,0,<class 'str'>
8,Hour,24,24,0,<class 'numpy.int64'>
9,Weekend,2,2,0,<class 'numpy.int64'>


In [57]:
#time values are all the columns present in the training set but not the test set (ones we are trying to predict I presume)
time_values = train.iloc[:,12:-1]
train_info.iloc[12:-1]
train_info[train_info.index.isin(list(time_values.columns))]
train_info.index

RangeIndex(start=0, stop=28, step=1)

In [5]:
# DistanceToFirstStop is how far before the intersection the truck stopped for the first time and
# TimeFromFirstStop is how long it took from that point to cross the intersection.
# I think that there should be a relation:
# TimeFromFirstStop=TotalTimeStopped + something like TotalDrivingTimeBetweenStops
# and we should expect that always:
# TimeFromFirstStop>TotalTimeStopped

In [17]:
timefromstop = train.iloc[:,12:17]
totaltimestopped = train.iloc[:,17:22]
distancefromstop = train.iloc[:,22:27]

In [62]:
train.IntersectionId.value_counts()

112     3422
270     2839
331     2770
335     2677
371     2366
        ... 
1579       1
2091       1
2850       1
2243       1
2776       1
Name: IntersectionId, Length: 2539, dtype: int64